# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7effb5e3a280>
├── 'a' --> tensor([[ 0.4287, -1.2416,  0.7783],
│                   [-1.0563,  1.7227, -0.5389]])
└── 'x' --> <FastTreeValue 0x7effb5e3af10>
    └── 'c' --> tensor([[ 0.3033, -0.2453,  0.2932, -0.1071],
                        [ 0.2052,  0.4708,  1.2985, -0.8520],
                        [ 0.5415,  0.6472, -0.4065, -0.6815]])

In [4]:
t.a

tensor([[ 0.4287, -1.2416,  0.7783],
        [-1.0563,  1.7227, -0.5389]])

In [5]:
%timeit t.a

74.5 ns ± 2.33 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7effb5e3a280>
├── 'a' --> tensor([[ 1.2447,  1.1150, -1.8343],
│                   [ 0.8144, -0.0576, -1.2549]])
└── 'x' --> <FastTreeValue 0x7effb5e3af10>
    └── 'c' --> tensor([[ 0.3033, -0.2453,  0.2932, -0.1071],
                        [ 0.2052,  0.4708,  1.2985, -0.8520],
                        [ 0.5415,  0.6472, -0.4065, -0.6815]])

In [7]:
%timeit t.a = new_value

85.3 ns ± 1.05 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.4287, -1.2416,  0.7783],
               [-1.0563,  1.7227, -0.5389]]),
    x: Batch(
           c: tensor([[ 0.3033, -0.2453,  0.2932, -0.1071],
                      [ 0.2052,  0.4708,  1.2985, -0.8520],
                      [ 0.5415,  0.6472, -0.4065, -0.6815]]),
       ),
)

In [10]:
b.a

tensor([[ 0.4287, -1.2416,  0.7783],
        [-1.0563,  1.7227, -0.5389]])

In [11]:
%timeit b.a

73.1 ns ± 2.59 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.4341,  1.6436, -1.0813],
               [ 0.7333,  2.5109,  0.0365]]),
    x: Batch(
           c: tensor([[ 0.3033, -0.2453,  0.2932, -0.1071],
                      [ 0.2052,  0.4708,  1.2985, -0.8520],
                      [ 0.5415,  0.6472, -0.4065, -0.6815]]),
       ),
)

In [13]:
%timeit b.a = new_value

656 ns ± 4.68 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.06 µs ± 22.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.5 µs ± 199 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

279 µs ± 19.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

257 µs ± 9.21 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7efee5f7c9d0>
├── 'a' --> tensor([[[ 0.4287, -1.2416,  0.7783],
│                    [-1.0563,  1.7227, -0.5389]],
│           
│                   [[ 0.4287, -1.2416,  0.7783],
│                    [-1.0563,  1.7227, -0.5389]],
│           
│                   [[ 0.4287, -1.2416,  0.7783],
│                    [-1.0563,  1.7227, -0.5389]],
│           
│                   [[ 0.4287, -1.2416,  0.7783],
│                    [-1.0563,  1.7227, -0.5389]],
│           
│                   [[ 0.4287, -1.2416,  0.7783],
│                    [-1.0563,  1.7227, -0.5389]],
│           
│                   [[ 0.4287, -1.2416,  0.7783],
│                    [-1.0563,  1.7227, -0.5389]],
│           
│                   [[ 0.4287, -1.2416,  0.7783],
│                    [-1.0563,  1.7227, -0.5389]],
│           
│                   [[ 0.4287, -1.2416,  0.7783],
│                    [-1.0563,  1.7227, -0.5389]]])
└── 'x' --> <FastTreeValue 0x7efee5f7c790>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

45.6 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7efee5f7cb50>
├── 'a' --> tensor([[ 0.4287, -1.2416,  0.7783],
│                   [-1.0563,  1.7227, -0.5389],
│                   [ 0.4287, -1.2416,  0.7783],
│                   [-1.0563,  1.7227, -0.5389],
│                   [ 0.4287, -1.2416,  0.7783],
│                   [-1.0563,  1.7227, -0.5389],
│                   [ 0.4287, -1.2416,  0.7783],
│                   [-1.0563,  1.7227, -0.5389],
│                   [ 0.4287, -1.2416,  0.7783],
│                   [-1.0563,  1.7227, -0.5389],
│                   [ 0.4287, -1.2416,  0.7783],
│                   [-1.0563,  1.7227, -0.5389],
│                   [ 0.4287, -1.2416,  0.7783],
│                   [-1.0563,  1.7227, -0.5389],
│                   [ 0.4287, -1.2416,  0.7783],
│                   [-1.0563,  1.7227, -0.5389]])
└── 'x' --> <FastTreeValue 0x7efee5f7cfd0>
    └── 'c' --> tensor([[ 0.3033, -0.2453,  0.2932, -0.1071],
                        [ 0.2052,  0.4708,  1.2985, -0.8520],
                 

In [23]:
%timeit t_cat(trees)

42.7 µs ± 744 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

96.7 µs ± 2.62 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.4287, -1.2416,  0.7783],
                [-1.0563,  1.7227, -0.5389]],
       
               [[ 0.4287, -1.2416,  0.7783],
                [-1.0563,  1.7227, -0.5389]],
       
               [[ 0.4287, -1.2416,  0.7783],
                [-1.0563,  1.7227, -0.5389]],
       
               [[ 0.4287, -1.2416,  0.7783],
                [-1.0563,  1.7227, -0.5389]],
       
               [[ 0.4287, -1.2416,  0.7783],
                [-1.0563,  1.7227, -0.5389]],
       
               [[ 0.4287, -1.2416,  0.7783],
                [-1.0563,  1.7227, -0.5389]],
       
               [[ 0.4287, -1.2416,  0.7783],
                [-1.0563,  1.7227, -0.5389]],
       
               [[ 0.4287, -1.2416,  0.7783],
                [-1.0563,  1.7227, -0.5389]]]),
    x: Batch(
           c: tensor([[[ 0.3033, -0.2453,  0.2932, -0.1071],
                       [ 0.2052,  0.4708,  1.2985, -0.8520],
                       [ 0.5415,  0.6472, -0.4065, -0.6815]],
         

In [26]:
%timeit Batch.stack(batches)

110 µs ± 2.05 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.4287, -1.2416,  0.7783],
               [-1.0563,  1.7227, -0.5389],
               [ 0.4287, -1.2416,  0.7783],
               [-1.0563,  1.7227, -0.5389],
               [ 0.4287, -1.2416,  0.7783],
               [-1.0563,  1.7227, -0.5389],
               [ 0.4287, -1.2416,  0.7783],
               [-1.0563,  1.7227, -0.5389],
               [ 0.4287, -1.2416,  0.7783],
               [-1.0563,  1.7227, -0.5389],
               [ 0.4287, -1.2416,  0.7783],
               [-1.0563,  1.7227, -0.5389],
               [ 0.4287, -1.2416,  0.7783],
               [-1.0563,  1.7227, -0.5389],
               [ 0.4287, -1.2416,  0.7783],
               [-1.0563,  1.7227, -0.5389]]),
    x: Batch(
           c: tensor([[ 0.3033, -0.2453,  0.2932, -0.1071],
                      [ 0.2052,  0.4708,  1.2985, -0.8520],
                      [ 0.5415,  0.6472, -0.4065, -0.6815],
                      [ 0.3033, -0.2453,  0.2932, -0.1071],
                      [ 0.2052,  

In [28]:
%timeit Batch.cat(batches)

196 µs ± 3.36 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

608 µs ± 25.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
